<a href="https://colab.research.google.com/github/dframbaut/web-scrapping-deepear/blob/main/web_scrapping_simi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 29.8 MB/s eta 0:00:00


In [4]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,192 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,626 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpad

In [5]:
!apt-get install -y chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 51 not upgraded.
Need to get 30.2 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.4 [598 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

In [6]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [9]:
# Configurar Selenium para Colab
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Configuración del navegador en modo headless
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Modo sin interfaz gráfica
options.add_argument('--no-sandbox')  # Necesario para Colab
options.add_argument('--disable-dev-shm-usage')  # Evitar problemas de memoria
options.add_argument('--disable-gpu')

# Inicializar el controlador del navegador
driver = webdriver.Chrome(options=options)

# URL base
url = "https://simi.concejodemedellin.gov.co/simi3/invitados/proyectos/acuerdos.xhtml"

# Inicializar listas para almacenar los datos
Nombres = []
Fechas = []
Epigrafe = []
Tipo = []
botones = []

try:
    # Navegar al sitio web
    driver.get(url)

    # Esperar a que la página cargue completamente
    time.sleep(5)

    while True:  # Bucle para recorrer todas las páginas
        print("Procesando página...")

        # Localizar la tabla en la página actual
        table = driver.find_element(By.TAG_NAME, "table")
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")
        # Localizar todos los botones con "ui-button"
        buttons = driver.find_elements(By.CLASS_NAME, "ui-button")  # buscar todos los botones

        # Procesar cada fila de la tabla
        for row_index, row in enumerate(rows, start=1):
            cells = row.find_elements(By.TAG_NAME, "td")
            for col_index, cell in enumerate(cells, start=1):
                cell_content = cell.text.strip()
                if col_index == 1:  # Nombre
                    Nombres.append(cell_content)
                elif col_index == 2:  # Fecha
                    Fechas.append(cell_content)
                elif col_index == 3:  # Epígrafe
                    Epigrafe.append(cell_content)
                elif col_index == 4:  # Tipo
                    Tipo.append(cell_content)
                elif col_index == 5:  # Enlace
                  for i, button in enumerate(buttons, start=1):
                    onclick_text = button.get_attribute("onclick")
                    if onclick_text:  # Solo agregar si el atributo 'onclick' no está vacío
                      botones.append(onclick_text)

        # Intentar hacer clic en el siguiente botón numérico
        try:
            # Buscar todos los botones de paginación
            page_buttons = driver.find_elements(By.CSS_SELECTOR, ".ui-paginator-page")#Busca todos los marcados por ".ui-paginator-page"

            # Buscar el siguiente botón de la paginación
            for button in page_buttons:
                if button.text and button.text.isdigit():
                    page_number = int(button.text)
                    if page_number > 1:  # Ignorar la página 1, ya que es la página inicial
                        button.click()
                        time.sleep(3)  # Esperar a que la página cargue
                        break
            if len(page_buttons) == 0 or page_buttons[-1].text == '10':  # Recorrer de la pagina 0 hasta la 10
                print("Última página alcanzada.")
                break

        except Exception as e:
            print("Error al navegar a la siguiente página:", e)
            break

    # Imprimir las listas de datos extraídos
    print("Nombres:", Nombres)
    print("Fechas:", Fechas)
    print("Epígrafe:", Epigrafe)
    print("Tipo:", Tipo)
    print("Botones:", botones)

finally:
    # Cerrar el navegador
    driver.quit()


Procesando página...
Error al navegar a la siguiente página: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.87); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x573d2b99434a <unknown>
#1 0x573d2b4aa6e0 <unknown>
#2 0x573d2b4b04a3 <unknown>
#3 0x573d2b4b2968 <unknown>
#4 0x573d2b4b29f3 <unknown>
#5 0x573d2b4f3ba2 <unknown>
#6 0x573d2b51d462 <unknown>
#7 0x573d2b4eda18 <unknown>
#8 0x573d2b51d62e <unknown>
#9 0x573d2b53bed7 <unknown>
#10 0x573d2b51d203 <unknown>
#11 0x573d2b4ebcc0 <unknown>
#12 0x573d2b4ecc9e <unknown>
#13 0x573d2b961d3b <unknown>
#14 0x573d2b965cc2 <unknown>
#15 0x573d2b94eb6c <unknown>
#16 0x573d2b966837 <unknown>
#17 0x573d2b93410f <unknown>
#18 0x573d2b9835a8 <unknown>
#19 0x573d2b983770 <unknown>
#20 0x573d2b9931c6 <unknown>
#21 0x79510666dac3 <unknown>

Nombres: ['16-2024', '14-2